In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#drop(5), ref
DROPREF = True


datafile = "../data/raw_data/COVID220510_mod.xlsx"
static_covid=pd.read_excel(datafile, sheet_name = '환자별 정보내역')
print(static_covid.columns)

vital_covid_mv=pd.read_excel(datafile, sheet_name = '인공호흡 임상관찰')
vital_covid_nmv=pd.read_excel(datafile, sheet_name = '호흡미처방 임상관찰')
CBC_covid_mv=pd.read_excel(datafile, sheet_name = '인공호흡 혈액(CBC)')
CBC_covid_nmv=pd.read_excel(datafile, sheet_name = '호흡미처방 혈액(CBC)')
ABGA_covid_mv=pd.read_excel(datafile, sheet_name = '인공호흡 혈액(Blood gas study)')
ABGA_covid_nmv=pd.read_excel(datafile, sheet_name = '호흡미처방 혈액(Blood gas study)')
print(ABGA_covid_mv.columns)

Index(['환자번호', '코로나병동입실시간', '병원퇴실시간', '나이', '성별', '사망여부', '인공호흡시작처방',
       '인공호흡종료일자', '인공호흡 처방일수', '인공호흡처방코드'],
      dtype='object')
Index(['환자', '날짜', 'ph', 'pco2', 'po2', 'hco3', 'bevv', 'spo2_lab', '환자.1',
       '날짜.1', 'fio2', 'pfratio'],
      dtype='object')


In [2]:
static_covid_ = static_covid[['환자번호','코로나병동입실시간','병원퇴실시간','나이','성별', '인공호흡시작처방', '인공호흡처방코드']].copy()
static_covid_.columns = ['pid','icu_in','icu_out','age','gender','MV_time','MV_code']
static_covid_

static_covid_['icu_in']=static_covid_['icu_in'].astype('datetime64')
static_covid_['icu_out']=static_covid_['icu_out'].astype('datetime64')
static_covid_['MV_time']= pd.to_datetime(static_covid_['MV_time'])
static_covid_.loc[:, 'intime'] = static_covid_['icu_in'].apply(lambda x : x.hour)

#filter row with mv code but no mv time ( 10 patients)
# == admission with mv 
static_covid_.loc[:,'IHMV']= ~pd.isnull(static_covid_['MV_time'])
static_covid_['MV_code']= ~pd.isnull(static_covid_['MV_code'])
static_covid_ = static_covid_[static_covid_['MV_code']==static_covid_['IHMV']]
static_covid_.drop(labels='MV_code',axis=1,inplace=True)
static_covid_['los'] = static_covid_['icu_out'] - static_covid_['icu_in']

t=~pd.isnull(static_covid_['MV_time'])
static_covid_['MV_time_'] = np.nan
static_covid_['MV_time_'][t] = (static_covid_['MV_time'][t] - static_covid_['icu_in'][t]).astype('timedelta64[h]')
static_covid_['MV_time_'][t]
static_covid = static_covid_

<ipython-input-2-5713c2a2c62d>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  static_covid_['MV_time_'][t] = (static_covid_['MV_time'][t] - static_covid_['icu_in'][t]).astype('timedelta64[h]')


In [3]:
static_to_keep = static_covid_[['pid','age','icu_in','intime','gender']].copy()
static_to_keep = pd.get_dummies(static_to_keep, columns = ['gender'])
static_to_keep.sort_values(by=['pid'],inplace = True)
static_to_keep['pid']=static_to_keep['pid'].round(0).astype(int)
static_to_keep

,pid,age,icu_in,intime,gender_F,gender_M
0,300003674,73,2021-11-13 17:49:00,17,0,1
1,300005919,40,2021-08-09 10:30:00,10,0,1
2,300023670,56,2021-03-15 16:08:00,16,0,1
3,300243763,55,2021-07-20 11:06:00,11,0,1
4,300298653,65,2021-09-24 13:27:00,13,0,1
...,...,...,...,...,...,...
146,302063527,60,2021-12-09 12:41:00,12,0,1
147,302064074,67,2021-12-12 16:47:00,16,1,0
148,302065492,70,2021-12-19 19:09:00,19,1,0
149,302066303,48,2021-12-25 12:11:00,12,1,0


In [4]:
print(vital_covid_mv.columns) 
print(CBC_covid_mv.columns)
print(ABGA_covid_mv.columns)

_vital = ['rr','hr','spo2','sbp','dbp','temp']
_CBC = ['hct','hb','platelets','wbc','rbc','mch','mchc','mcv']
_ABGA = ['ph','hco3', 'po2','pco2','sao2','fio2','pfratio']
cols = _vital+_CBC+_ABGA
ID_COLS = ['pid','date']

Index(['환자', '날짜', 'SBP', '날짜.1', 'DBP', '환자.1', '날짜.2', 'BT', '환자.2', '날짜.3',
       'RR', '날짜.4', 'HR', '날짜.5', 'SpO2'],
      dtype='object')
Index(['환자', '날짜', 'hb', 'hct', 'rbc', 'mcv', 'mch', 'mchc', 'wbc',
       'platelets'],
      dtype='object')
Index(['환자', '날짜', 'ph', 'pco2', 'po2', 'hco3', 'bevv', 'spo2_lab', '환자.1',
       '날짜.1', 'fio2', 'pfratio'],
      dtype='object')


In [5]:
# CBC processing
#_CBC = ['hct','hb','platelets','wbc','rbc','mch','mchc','mcv']
CBC_covid = pd.concat([CBC_covid_mv,CBC_covid_nmv])
print(CBC_covid.shape)
CBC_covid.rename(columns = {'환자':'pid','날짜':'date'}, inplace = True)
CBC_covid['date']=pd.to_datetime(CBC_covid['date'])
#CBC_covid.dropna(subset=['pid'],inplace=True)
CBC_covid.sort_values(by=['pid','date'],inplace=True)
CBC_covid=CBC_covid[ID_COLS + _CBC]
CBC_covid

(950, 10)


,pid,date,hct,hb,platelets,wbc,rbc,mch,mchc,mcv
0,300003674,2021-11-14 05:29:27,44.4,15.2,206,2.37,4.91,31.0,34.2,90.4
1,300003674,2021-11-15 06:20:48,44.2,14.8,216,3.47,4.86,30.5,33.5,90.9
2,300003674,2021-11-16 05:31:54,43.9,15.0,272,6.00,4.87,30.8,34.2,90.1
3,300003674,2021-11-17 04:58:56,41.1,14.2,261,7.94,4.60,30.9,34.5,89.3
4,300003674,2021-11-18 06:04:09,42.4,14.7,258,10.15,4.74,31.0,34.7,89.5
...,...,...,...,...,...,...,...,...,...,...
709,302066555,2021-12-25 05:54:15,38.3,13.6,215,14.26,4.28,31.8,35.5,89.5
710,302066555,2021-12-26 05:23:34,35.3,12.5,180,10.51,3.89,32.1,35.4,90.7
711,302066555,2021-12-27 05:52:27,35.2,12.0,175,7.57,3.74,32.1,34.1,94.1
712,302066555,2021-12-29 05:11:03,35.8,12.1,193,6.18,3.84,31.5,33.8,93.2


In [6]:
# ABGA processing
#_ABGA = ['ph','hco3', 'po2','pco2','spo2_lab','fio2','pfratio']
ABGA_covid = pd.concat([ABGA_covid_mv,ABGA_covid_nmv])
ABGA_a = ABGA_covid.iloc[:,0:8].copy()
ABGA_b = ABGA_covid.iloc[:,8:].copy()

ABGA = [ABGA_a,ABGA_b]
print(ABGA_covid.shape)
for vt in ABGA:
    vt.rename(columns = { i: 'pid' for i in  vt.columns if i.startswith('환자') }, inplace=True )
    vt.rename(columns = { i: 'date' for i in  vt.columns if i.startswith('날짜') }, inplace=True )
    vt['date']=pd.to_datetime(vt['date'])
    vt.dropna(subset=['pid'],inplace=True)
    vt['pid']=vt['pid'].round(0).astype(int)
    print(vt.dtypes)
ABGA_covid = pd.merge(left = ABGA[0] , right = ABGA[1], how = "outer", on=['pid','date'])
ABGA_covid.sort_values(by=['pid','date'],inplace=True)
ABGA_covid.drop('bevv', axis=1, inplace=True)

#_ABGA = ['ph','hco3', 'po2','pco2','sao2','fio2']
ABGA_covid.rename(columns={'spo2_lab':'sao2'}, inplace = True)
ABGA_covid[_ABGA] = ABGA_covid[_ABGA].astype(float)
ABGA_covid = ABGA_covid[ID_COLS+_ABGA]
ABGA_covid

(2204, 12)
pid                  int64
date        datetime64[ns]
ph                 float64
pco2               float64
po2                float64
hco3               float64
bevv               float64
spo2_lab           float64
dtype: object
pid                 int64
date       datetime64[ns]
fio2              float64
pfratio           float64
dtype: object


,pid,date,ph,hco3,po2,pco2,sao2,fio2,pfratio
0,300003674,2021-11-13 19:48:46,7.409,19.9,51.3,31.1,86.8,90.0,57.0
1,300003674,2021-11-13 22:11:20,7.440,19.3,66.4,28.1,93.9,90.0,73.8
2,300003674,2021-11-14 03:59:54,7.433,19.5,60.3,28.9,91.9,90.0,67.0
3,300003674,2021-11-14 10:06:32,7.474,19.4,80.9,26.1,96.9,90.0,89.9
4,300003674,2021-11-14 13:39:53,7.474,20.7,177.8,28.0,99.7,NaN,NaN
...,...,...,...,...,...,...,...,...,...
2201,302064074,2021-12-22 16:46:42,7.454,25.5,80.0,36.0,96.4,24.0,333.5
2202,302064074,2021-12-23 04:51:58,7.468,23.2,69.7,31.7,95.0,24.0,290.3
2203,302064074,2021-12-23 16:59:20,7.476,22.7,87.0,30.6,97.4,24.0,362.6
946,302065492,2021-12-19 21:25:05,7.403,22.0,101.7,34.9,97.9,95.0,107.1


In [7]:
#vital preprocessing
#_vital = ['rr','hr','spo2','sbp','dbp','temp']

vital_covid = pd.concat([vital_covid_mv,vital_covid_nmv]) 
vital_covid
v1 = vital_covid.iloc[:,[0,1,2,4]].dropna(how='any')
print(v1.shape)
v2 = vital_covid.iloc[:,[5,6,7]].dropna(how='any')
print(v2.shape)
v3 = vital_covid.iloc[:,[8,9,10,12,14]].dropna(how='any')
print(v3.shape)

vital = [v1,v2,v3]

for idx, vt in enumerate(vital):
    vt.rename(columns = { i: 'pid' for i in  vt.columns if i.startswith('환자') }, inplace=True )
    vt.rename(columns = { i: 'date' for i in  vt.columns if i.startswith('날짜') }, inplace=True )
    vt['pid'] = vt['pid'].astype('int64')
    vt['date'] = vt['date'].astype('datetime64')
    #vt.iloc[:, 2] = vt.iloc[:, 2].astype('int64')
    vital[idx] = vt.sort_values(by=['pid','date'])
    print(vt.dtypes)

vital_covid = pd.merge(left = v1 , right = v2, how = "outer", on=['pid','date'])
print(vital_covid.shape)
vital_covid = pd.merge(left = vital_covid , right = v3, how = "outer", on=['pid','date'])
print(vital_covid.shape)
vital_covid.columns = ['pid','date','sbp','dbp','temp','rr','hr','spo2']
vital_covid = vital_covid[ID_COLS+_vital]

vital_covid

(16298, 4)
(14741, 3)
(16770, 5)
pid              int64
date    datetime64[ns]
SBP            float64
DBP            float64
dtype: object
pid              int64
date    datetime64[ns]
BT             float64
dtype: object
pid              int64
date    datetime64[ns]
RR               int64
HR               int64
SpO2             int64
dtype: object
(17004, 5)
(17660, 8)


,pid,date,rr,hr,spo2,sbp,dbp,temp
0,300003674,2021-11-13 17:00:00,18.0,92.0,98.0,137.0,76.0,NaN
1,300003674,2021-11-13 18:00:00,25.0,91.0,98.0,148.0,73.0,NaN
2,300003674,2021-11-13 19:10:00,25.0,85.0,97.0,139.0,74.0,37.4
3,300003674,2021-11-13 20:00:00,22.0,89.0,92.0,138.0,76.0,37.3
4,300003674,2021-11-13 21:00:00,16.0,96.0,96.0,160.0,87.0,37.5
...,...,...,...,...,...,...,...,...
17655,302066303,2022-01-03 04:00:00,21.0,91.0,93.0,NaN,NaN,NaN
17656,302066303,2022-01-03 15:00:00,16.0,67.0,98.0,NaN,NaN,NaN
17657,302066303,2022-01-04 15:00:00,23.0,80.0,95.0,NaN,NaN,NaN
17658,302066555,2021-12-24 13:00:00,17.0,60.0,99.0,NaN,NaN,NaN


In [8]:
#merge lab & vital data
X = pd.merge(left = vital_covid, right = CBC_covid, how = "outer", on=['pid','date'])
X = pd.merge(left = X, right = ABGA_covid, how = "outer", on=['pid','date'])
X.sort_values(by=['pid','date'],inplace=True)
X=X.reset_index(drop=True)
X = pd.merge(left = static_to_keep[['pid','icu_in']], right = X, how = "inner", on='pid')

#date to hours_in
X.insert(1,'hours_in','')
X['hours_in'] = (X['date'] - X['icu_in']).astype('timedelta64[h]')+1
X.drop(['icu_in','date'], axis=1, inplace=True)
X

,pid,hours_in,rr,hr,spo2,sbp,dbp,temp,hct,hb,...,mch,mchc,mcv,ph,hco3,po2,pco2,sao2,fio2,pfratio
0,300003674,0.0,18.0,92.0,98.0,137.0,76.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,300003674,1.0,25.0,91.0,98.0,148.0,73.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,300003674,2.0,25.0,85.0,97.0,139.0,74.0,37.4,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,300003674,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,7.409,19.9,51.3,31.1,86.8,90.0,57.0
4,300003674,3.0,22.0,89.0,92.0,138.0,76.0,37.3,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21471,302066555,222.0,14.0,63.0,100.0,92.0,65.0,36.2,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21472,302066555,223.0,16.0,54.0,99.0,84.0,59.0,37.2,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21473,302066555,224.0,NaN,NaN,NaN,83.0,50.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21474,302066555,224.0,17.0,58.0,99.0,82.0,55.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
# save raw data (no imputation & no outlier removal)
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

X_raw= X.copy()
X_raw[_vital].boxplot()


<AxesSubplot:>

In [10]:
#delete outlier

#_vital = ['rr','hr','spo2','sbp','dbp','temp']
#_CBC = ['hct','hb','platelets','wbc','rbc','mch','mchc','mcv']
#_ABGA = ['ph','hco3', 'po2','pco2','sao2','fio2','pfratio']
#cols = _vital+_CBC+_ABGA

outlier_range = {}
outlier_range['rr']=[0,375]
outlier_range['hr']=[0,390]
outlier_range['spo2']=[0,150]
outlier_range['sbp']=[0,375]
outlier_range['dbp']=[0,375]
outlier_range['temp']=[14.2,47]

outlier_range['hct']=[0,100]
outlier_range['hb']=[0,30]
outlier_range['platelets']=[0,2200]
outlier_range['wbc']=[0,1100]

outlier_range['ph']=[0,10]
outlier_range['po2']=[0,770]
outlier_range['pco2']=[0,220]
outlier_range['sao2']=[0,150]
outlier_range['fio2']=[20,110]

for i in outlier_range.keys():
    X.mask(X[i]<outlier_range[i][0], inplace = True)
    X.mask(X[i]>outlier_range[i][1], inplace = True)
# change fio2 scale
X['fio2'] = X['fio2']/100
X.describe()

,pid,hours_in,rr,hr,spo2,sbp,dbp,temp,hct,hb,...,mch,mchc,mcv,ph,hco3,po2,pco2,sao2,fio2,pfratio
count,2.147200e+04,21472.000000,17487.000000,17487.000000,17487.000000,16967.000000,16967.000000,15545.000000,975.000000,975.000000,...,975.000000,975.000000,975.000000,2206.000000,2203.000000,2216.000000,2214.000000,2201.000000,2205.000000,2207.000000
mean,3.015506e+08,205.148053,20.153028,75.313204,96.182764,123.509283,74.679554,36.724555,34.132205,11.464615,...,30.633846,33.578359,91.228821,7.426142,21.530640,87.685605,32.781301,95.279918,0.521974,203.688899
std,6.096989e+05,471.294566,5.198411,18.357014,3.996180,20.226034,13.172552,0.526506,6.810747,2.330071,...,2.083499,1.041339,5.610395,0.069007,3.698836,29.303755,8.593812,4.354809,0.213701,118.509144
min,3.000037e+08,-1088.000000,0.000000,0.000000,0.000000,0.000000,0.000000,26.800000,18.700000,6.000000,...,24.100000,29.900000,74.600000,6.766000,10.000000,28.800000,15.400000,48.600000,0.200000,34.700000
25%,3.010096e+08,37.000000,17.000000,62.000000,95.000000,110.000000,65.000000,36.400000,28.800000,9.600000,...,29.300000,32.900000,87.300000,7.409000,19.100000,68.200000,27.600000,93.900000,0.350000,114.800000
50%,3.019719e+08,99.000000,20.000000,73.000000,97.000000,123.000000,74.000000,36.700000,34.900000,11.700000,...,30.500000,33.500000,91.400000,7.439000,21.300000,81.650000,31.500000,96.400000,0.500000,176.200000
75%,3.020474e+08,176.000000,23.000000,86.000000,99.000000,137.000000,83.000000,37.000000,38.900000,13.200000,...,31.900000,34.200000,94.800000,7.465000,23.800000,99.600000,36.200000,98.000000,0.650000,264.600000
max,3.020666e+08,3199.000000,73.000000,197.000000,100.000000,367.000000,179.000000,41.000000,50.200000,17.400000,...,39.200000,38.200000,107.000000,7.627000,43.000000,308.600000,200.000000,99.900000,1.000000,851.400000


In [11]:
reference_value = {      # average
#vital
    'rr': 16,           # 19.75339
    'hr':80,            # 86.15997
    'spo2': 97,      # 90.743683
    'sbp':110,          # 122.7326
    'dbp':70,           # 60.76322
    'temp':36.5,        # 37.0867
#CBC
    'hct': 41.5,        # 30.491288
    'hb': 13.5,         # 10.396213
    'platelets':275,    # 219.158934
    'wbc': 7,           # 12.412393
    'rbc': 4.45,        # 3.454153
    'mch': 29.5,        # 30.333704
    'mchc': 33.25,      # 33.850937
    'mcv': 90,          # 89.702025
#ABGA
    'ph': 7.4,          # 7.388153
    'hco3': 24,         # 24.595758
    'po2': 87.5,        # 130.492106
    'pco2': 40,         # 41.616208
    'sao2': 97,     # 97.09498
    'fio2' : 0.21       # 0.512136
}

#extract aggregate value per hour
X_ = X.groupby(['pid'] + ['hours_in']).agg(['mean', 'first','min','max'])
print(X.columns)

print(X_.isna().sum().sum())

if DROPREF:
    print("DROPREF")
    print(X_.shape)
    print(X_.isna().sum().sum())
    X_=X_.groupby(['pid'] ).fillna(method='ffill')
    print(X_.isna().sum().sum())
    X_=X_.dropna(axis=0,thresh=23-5)
    print(X_.isna().sum().sum())
    print(X_.shape)
else:
    X_=X_.groupby(['pid'] ).fillna(method='ffill').groupby(['pid'] ).fillna(method='bfill')
#X_=X_.groupby(['pid'] ).fillna(method='ffill').groupby(['pid'] ).fillna(method='bfill',limit=6)
#X_=X_.groupby(['pid'] ).fillna(method='ffill')
print(X_.isna().sum().sum())
X_.fillna(
    pd.Series(X_.columns.get_level_values(0).map(reference_value.get), X_.columns), inplace=True
)
print(X_.isna().sum().sum())
X_

Index(['pid', 'hours_in', 'rr', 'hr', 'spo2', 'sbp', 'dbp', 'temp', 'hct',
       'hb', 'platelets', 'wbc', 'rbc', 'mch', 'mchc', 'mcv', 'ph', 'hco3',
       'po2', 'pco2', 'sao2', 'fio2', 'pfratio'],
      dtype='object')
1025024
DROPREF
(18000, 84)
1025024
103508
97200
(17914, 84)
97200
7772


rr                      hr                     spo2  \
                      mean first   min   max  mean first   min   max   mean   
pid         hours_in                                                          
300003674.0 0.0       18.0  18.0  18.0  18.0  92.0  92.0  92.0  92.0   98.0   
            1.0       25.0  25.0  25.0  25.0  91.0  91.0  91.0  91.0   98.0   
            2.0       25.0  25.0  25.0  25.0  85.0  85.0  85.0  85.0   97.0   
            3.0       22.0  22.0  22.0  22.0  89.0  89.0  89.0  89.0   92.0   
            4.0       16.0  16.0  16.0  16.0  96.0  96.0  96.0  96.0   96.0   
...                    ...   ...   ...   ...   ...   ...   ...   ...    ...   
302066555.0 221.0     20.0  20.0  20.0  20.0  71.0  71.0  71.0  71.0   97.0   
            222.0     14.0  14.0  14.0  14.0  63.0  63.0  63.0  63.0  100.0   
            223.0     16.0  16.0  16.0  16.0  54.0  54.0  54.0  54.0   99.0   
            224.0     17.0  17.0  17.0  17.0  58.0  58.0  58.0  58.0   99.0   
            225.0     17.0  17.0  17.0  17.0  65.0  65.0  65.0  65.0  100.0   

                             ...  sao2        fio2                   pfratio  \
                      first  ...   min   max  mean first   min   max    mean   
pid         hours_in         ...                                               
300003674.0 0.0        98.0  ...  97.0  97.0  0.21  0.21  0.21  0.21     NaN   
            1.0        98.0  ...  97.0  97.0  0.21  0.21  0.21  0.21     NaN   
            2.0        97.0  ...  86.8  86.8  0.90  0.90  0.90  0.90    57.0   
            3.0        92.0  ...  86.8  86.8  0.90  0.90  0.90  0.90    57.0   
            4.0        96.0  ...  86.8  86.8  0.90  0.90  0.90  0.90    57.0   
...                     ...  ...   ...   ...   ...   ...   ...   ...     ...   
302066555.0 221.0      97.0  ...  97.0  97.0  0.21  0.21  0.21  0.21     NaN   
            222.0     100.0  ...  97.0  97.0  0.21  0.21  0.21  0.21     NaN   
            223.0      99.0  ...  97.0  97.0  0.21  0.21  0.21  0.21     NaN   
            224.0      99.0  ...  97.0  97.0  0.21  0.21  0.21  0.21     NaN   
            225.0     100.0  ...  97.0  97.0  0.21  0.21  0.21  0.21     NaN   

                                        
                     first   min   max  
pid         hours_in                    
300003674.0 0.0        NaN   NaN   NaN  
            1.0        NaN   NaN   NaN  
            2.0       57.0  57.0  57.0  
            3.0       57.0  57.0  57.0  
            4.0       57.0  57.0  57.0  
...                    ...   ...   ...  
302066555.0 221.0      NaN   NaN   NaN  
            222.0      NaN   NaN   NaN  
            223.0      NaN   NaN   NaN  
            224.0      NaN   NaN   NaN  
            225.0      NaN   NaN   NaN  

[17914 rows x 84 columns]

In [12]:
X_=X_.reset_index()
X_ = X_[X_['hours_in']>=0] # remove observation before ICU admission
X_ = X_[X_['hours_in']<=240] # maximum, 10 days of stay

# filter patients with earlyexit ==> 7 patients removed
temp = X_
print(len(temp['pid'].reset_index(drop=True).unique()))
temp = temp.reset_index(drop=True).groupby('pid').filter(lambda x: (x.hours_in >= 6).any() )
print(len(temp['pid'].unique()))
X_ = temp 
X_ = X_.set_index(['pid','hours_in'])
X_


141
134


rr                      hr                     spo2  \
                      mean first   min   max  mean first   min   max   mean   
pid         hours_in                                                          
300003674.0 0.0       18.0  18.0  18.0  18.0  92.0  92.0  92.0  92.0   98.0   
            1.0       25.0  25.0  25.0  25.0  91.0  91.0  91.0  91.0   98.0   
            2.0       25.0  25.0  25.0  25.0  85.0  85.0  85.0  85.0   97.0   
            3.0       22.0  22.0  22.0  22.0  89.0  89.0  89.0  89.0   92.0   
            4.0       16.0  16.0  16.0  16.0  96.0  96.0  96.0  96.0   96.0   
...                    ...   ...   ...   ...   ...   ...   ...   ...    ...   
302066555.0 221.0     20.0  20.0  20.0  20.0  71.0  71.0  71.0  71.0   97.0   
            222.0     14.0  14.0  14.0  14.0  63.0  63.0  63.0  63.0  100.0   
            223.0     16.0  16.0  16.0  16.0  54.0  54.0  54.0  54.0   99.0   
            224.0     17.0  17.0  17.0  17.0  58.0  58.0  58.0  58.0   99.0   
            225.0     17.0  17.0  17.0  17.0  65.0  65.0  65.0  65.0  100.0   

                             ...  sao2        fio2                   pfratio  \
                      first  ...   min   max  mean first   min   max    mean   
pid         hours_in         ...                                               
300003674.0 0.0        98.0  ...  97.0  97.0  0.21  0.21  0.21  0.21     NaN   
            1.0        98.0  ...  97.0  97.0  0.21  0.21  0.21  0.21     NaN   
            2.0        97.0  ...  86.8  86.8  0.90  0.90  0.90  0.90    57.0   
            3.0        92.0  ...  86.8  86.8  0.90  0.90  0.90  0.90    57.0   
            4.0        96.0  ...  86.8  86.8  0.90  0.90  0.90  0.90    57.0   
...                     ...  ...   ...   ...   ...   ...   ...   ...     ...   
302066555.0 221.0      97.0  ...  97.0  97.0  0.21  0.21  0.21  0.21     NaN   
            222.0     100.0  ...  97.0  97.0  0.21  0.21  0.21  0.21     NaN   
            223.0      99.0  ...  97.0  97.0  0.21  0.21  0.21  0.21     NaN   
            224.0      99.0  ...  97.0  97.0  0.21  0.21  0.21  0.21     NaN   
            225.0     100.0  ...  97.0  97.0  0.21  0.21  0.21  0.21     NaN   

                                        
                     first   min   max  
pid         hours_in                    
300003674.0 0.0        NaN   NaN   NaN  
            1.0        NaN   NaN   NaN  
            2.0       57.0  57.0  57.0  
            3.0       57.0  57.0  57.0  
            4.0       57.0  57.0  57.0  
...                    ...   ...   ...  
302066555.0 221.0      NaN   NaN   NaN  
            222.0      NaN   NaN   NaN  
            223.0      NaN   NaN   NaN  
            224.0      NaN   NaN   NaN  
            225.0      NaN   NaN   NaN  

[15165 rows x 84 columns]

In [13]:
idx = pd.IndexSlice
X_worst = X_.loc[:,idx[:, ['min','max']]].copy()
cnt=0
for k in X_worst.columns:
    if k[0] == 'pfratio':
        X_worst[k[0],'worst'] = X_worst[k[0],'min']
        continue
    #for min, max calc distance from reference value
    cnt=cnt+1
    if cnt%2==0:
        continue
    
    ref = reference_value[k[0]]
    print(k,cnt,ref)
    min_diff = abs(X_worst[k[0],'min'] - reference_value[k[0]] )
    max_diff = abs(X_worst[k[0],'max'] - reference_value[k[0]] )
    
    X_worst[k[0],'worst'] = np.where(max_diff>min_diff,
                    X_worst[k[0],'max'], X_worst[k[0],'min'])

X_worst = X_worst.loc[:,idx[:, ['worst']]]
X_worst.columns = X_worst.columns.droplevel(-1)
X_worst

('rr', 'min') 1 16
('hr', 'min') 3 80
('spo2', 'min') 5 97
('sbp', 'min') 7 110
('dbp', 'min') 9 70
('temp', 'min') 11 36.5
('hct', 'min') 13 41.5
('hb', 'min') 15 13.5
('platelets', 'min') 17 275
('wbc', 'min') 19 7
('rbc', 'min') 21 4.45
('mch', 'min') 23 29.5
('mchc', 'min') 25 33.25
('mcv', 'min') 27 90
('ph', 'min') 29 7.4
('hco3', 'min') 31 24
('po2', 'min') 33 87.5
('pco2', 'min') 35 40
('sao2', 'min') 37 97
('fio2', 'min') 39 0.21


rr    hr   spo2    sbp   dbp  temp   hct    hb  \
pid         hours_in                                                     
300003674.0 0.0       18.0  92.0   98.0  137.0  76.0  36.5  41.5  13.5   
            1.0       25.0  91.0   98.0  148.0  73.0  36.5  41.5  13.5   
            2.0       25.0  85.0   97.0  139.0  74.0  37.4  41.5  13.5   
            3.0       22.0  89.0   92.0  138.0  76.0  37.3  41.5  13.5   
            4.0       16.0  96.0   96.0  160.0  87.0  37.5  41.5  13.5   
...                    ...   ...    ...    ...   ...   ...   ...   ...   
302066555.0 221.0     20.0  71.0   97.0   89.0  60.0  36.9  37.3  12.8   
            222.0     14.0  63.0  100.0   92.0  65.0  36.2  37.3  12.8   
            223.0     16.0  54.0   99.0   84.0  59.0  37.2  37.3  12.8   
            224.0     17.0  58.0   99.0   82.0  50.0  37.2  37.3  12.8   
            225.0     17.0  65.0  100.0   83.0  57.0  37.2  37.3  12.8   

                      platelets   wbc  ...   mch   mchc   mcv     ph  hco3  \
pid         hours_in                   ...                                   
300003674.0 0.0           275.0  7.00  ...  29.5  33.25  90.0  7.400  24.0   
            1.0           275.0  7.00  ...  29.5  33.25  90.0  7.400  24.0   
            2.0           275.0  7.00  ...  29.5  33.25  90.0  7.409  19.9   
            3.0           275.0  7.00  ...  29.5  33.25  90.0  7.409  19.9   
            4.0           275.0  7.00  ...  29.5  33.25  90.0  7.409  19.9   
...                         ...   ...  ...   ...    ...   ...    ...   ...   
302066555.0 221.0         225.0  6.41  ...  31.9  34.30  93.0  7.400  24.0   
            222.0         225.0  6.41  ...  31.9  34.30  93.0  7.400  24.0   
            223.0         225.0  6.41  ...  31.9  34.30  93.0  7.400  24.0   
            224.0         225.0  6.41  ...  31.9  34.30  93.0  7.400  24.0   
            225.0         225.0  6.41  ...  31.9  34.30  93.0  7.400  24.0   

                       po2  pco2  sao2  fio2  pfratio  
pid         hours_in                                   
300003674.0 0.0       87.5  40.0  97.0  0.21      NaN  
            1.0       87.5  40.0  97.0  0.21      NaN  
            2.0       51.3  31.1  86.8  0.90     57.0  
            3.0       51.3  31.1  86.8  0.90     57.0  
            4.0       51.3  31.1  86.8  0.90     57.0  
...                    ...   ...   ...   ...      ...  
302066555.0 221.0     87.5  40.0  97.0  0.21      NaN  
            222.0     87.5  40.0  97.0  0.21      NaN  
            223.0     87.5  40.0  97.0  0.21      NaN  
            224.0     87.5  40.0  97.0  0.21      NaN  
            225.0     87.5  40.0  97.0  0.21      NaN  

[15165 rows x 21 columns]

In [14]:
#split dataset --> mean / first / worst
def split_data(X,target):
    idx = pd.IndexSlice
    X_=X.loc[:,idx[cols, target]]
    X_.columns = X_.columns.droplevel(-1)
    X_.columns.name = None
    return X_
X_mean = split_data(X_,'mean')
X_first = split_data(X_,'first')

dataset = [X_worst,X_mean,X_first]

In [15]:
static_to_keep.drop(columns='icu_in',inplace=True)

In [16]:
#fillin pfratio
for idx, df in enumerate(dataset):
    # fillin pfratio
    df.loc[:,'pfratio'] = np.where(df.pfratio.isnull(), df.po2/df.fio2, df.pfratio )
    dataset[idx]=df
dataset[0]

<ipython-input-16-d6f4e47d3dfd>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:,'pfratio'] = np.where(df.pfratio.isnull(), df.po2/df.fio2, df.pfratio )
<ipython-input-16-d6f4e47d3dfd>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:,'pfratio'] = np.where(df.pfratio.isnull(), df.po2/df.fio2, df.pfratio )


rr    hr   spo2    sbp   dbp  temp   hct    hb  \
pid         hours_in                                                     
300003674.0 0.0       18.0  92.0   98.0  137.0  76.0  36.5  41.5  13.5   
            1.0       25.0  91.0   98.0  148.0  73.0  36.5  41.5  13.5   
            2.0       25.0  85.0   97.0  139.0  74.0  37.4  41.5  13.5   
            3.0       22.0  89.0   92.0  138.0  76.0  37.3  41.5  13.5   
            4.0       16.0  96.0   96.0  160.0  87.0  37.5  41.5  13.5   
...                    ...   ...    ...    ...   ...   ...   ...   ...   
302066555.0 221.0     20.0  71.0   97.0   89.0  60.0  36.9  37.3  12.8   
            222.0     14.0  63.0  100.0   92.0  65.0  36.2  37.3  12.8   
            223.0     16.0  54.0   99.0   84.0  59.0  37.2  37.3  12.8   
            224.0     17.0  58.0   99.0   82.0  50.0  37.2  37.3  12.8   
            225.0     17.0  65.0  100.0   83.0  57.0  37.2  37.3  12.8   

                      platelets   wbc  ...   mch   mchc   mcv     ph  hco3  \
pid         hours_in                   ...                                   
300003674.0 0.0           275.0  7.00  ...  29.5  33.25  90.0  7.400  24.0   
            1.0           275.0  7.00  ...  29.5  33.25  90.0  7.400  24.0   
            2.0           275.0  7.00  ...  29.5  33.25  90.0  7.409  19.9   
            3.0           275.0  7.00  ...  29.5  33.25  90.0  7.409  19.9   
            4.0           275.0  7.00  ...  29.5  33.25  90.0  7.409  19.9   
...                         ...   ...  ...   ...    ...   ...    ...   ...   
302066555.0 221.0         225.0  6.41  ...  31.9  34.30  93.0  7.400  24.0   
            222.0         225.0  6.41  ...  31.9  34.30  93.0  7.400  24.0   
            223.0         225.0  6.41  ...  31.9  34.30  93.0  7.400  24.0   
            224.0         225.0  6.41  ...  31.9  34.30  93.0  7.400  24.0   
            225.0         225.0  6.41  ...  31.9  34.30  93.0  7.400  24.0   

                       po2  pco2  sao2  fio2     pfratio  
pid         hours_in                                      
300003674.0 0.0       87.5  40.0  97.0  0.21  416.666667  
            1.0       87.5  40.0  97.0  0.21  416.666667  
            2.0       51.3  31.1  86.8  0.90   57.000000  
            3.0       51.3  31.1  86.8  0.90   57.000000  
            4.0       51.3  31.1  86.8  0.90   57.000000  
...                    ...   ...   ...   ...         ...  
302066555.0 221.0     87.5  40.0  97.0  0.21  416.666667  
            222.0     87.5  40.0  97.0  0.21  416.666667  
            223.0     87.5  40.0  97.0  0.21  416.666667  
            224.0     87.5  40.0  97.0  0.21  416.666667  
            225.0     87.5  40.0  97.0  0.21  416.666667  

[15165 rows x 21 columns]

In [17]:
#apply merge
for idx, df in enumerate(dataset):
    dataset[idx]= pd.merge(df.reset_index(), static_to_keep, on='pid')
dataset[0].head()

,pid,hours_in,rr,hr,spo2,sbp,dbp,temp,hct,hb,...,hco3,po2,pco2,sao2,fio2,pfratio,age,intime,gender_F,gender_M
0,300003674.0,0.0,18.0,92.0,98.0,137.0,76.0,36.5,41.5,13.5,...,24.0,87.5,40.0,97.0,0.21,416.666667,73,17,0,1
1,300003674.0,1.0,25.0,91.0,98.0,148.0,73.0,36.5,41.5,13.5,...,24.0,87.5,40.0,97.0,0.21,416.666667,73,17,0,1
2,300003674.0,2.0,25.0,85.0,97.0,139.0,74.0,37.4,41.5,13.5,...,19.9,51.3,31.1,86.8,0.90,57.000000,73,17,0,1
3,300003674.0,3.0,22.0,89.0,92.0,138.0,76.0,37.3,41.5,13.5,...,19.9,51.3,31.1,86.8,0.90,57.000000,73,17,0,1
4,300003674.0,4.0,16.0,96.0,96.0,160.0,87.0,37.5,41.5,13.5,...,19.9,51.3,31.1,86.8,0.90,57.000000,73,17,0,1


In [18]:
# add absolute time feature
for idx, df in enumerate(dataset):
    abs_time = (df['intime'] + df['hours_in'])%24
    df.insert(2, 'absolute_time', abs_time)
    df.drop('intime', axis=1, inplace=True)
    dataset[idx]=df
dataset[0].head()

,pid,hours_in,absolute_time,rr,hr,spo2,sbp,dbp,temp,hct,...,ph,hco3,po2,pco2,sao2,fio2,pfratio,age,gender_F,gender_M
0,300003674.0,0.0,17.0,18.0,92.0,98.0,137.0,76.0,36.5,41.5,...,7.400,24.0,87.5,40.0,97.0,0.21,416.666667,73,0,1
1,300003674.0,1.0,18.0,25.0,91.0,98.0,148.0,73.0,36.5,41.5,...,7.400,24.0,87.5,40.0,97.0,0.21,416.666667,73,0,1
2,300003674.0,2.0,19.0,25.0,85.0,97.0,139.0,74.0,37.4,41.5,...,7.409,19.9,51.3,31.1,86.8,0.90,57.000000,73,0,1
3,300003674.0,3.0,20.0,22.0,89.0,92.0,138.0,76.0,37.3,41.5,...,7.409,19.9,51.3,31.1,86.8,0.90,57.000000,73,0,1
4,300003674.0,4.0,21.0,16.0,96.0,96.0,160.0,87.0,37.5,41.5,...,7.409,19.9,51.3,31.1,86.8,0.90,57.000000,73,0,1


In [19]:
for i in range(3):
    print(len(dataset[i]['pid'].unique()))

134
134
134


In [20]:
# remove static_covid with no data(X)
print(len(dataset[0]['pid'].unique()))
print(len(static_to_keep['pid'].unique()))
print(len(static_covid['pid'].unique()))

pid_list = dataset[0]['pid'].unique()
static_to_keep=static_to_keep[static_to_keep['pid'].isin(pid_list)]
static_covid=static_covid[static_covid['pid'].isin(pid_list)]

print(len(static_to_keep['pid'].unique()))
print(len(static_covid['pid'].unique()))

134
150
150
134
134


In [21]:
#filter observation after MV
for idx, df in enumerate(dataset):
    for subject_id in static_covid[static_covid['IHMV']==True]['pid']:
        vent_time = static_covid[static_covid['pid']==subject_id]['MV_time_'].item()
        if(vent_time <6):
            df = df.loc[df['pid']!=subject_id]
        else:
            df = df.drop(df[(df['pid']==subject_id) & (df['hours_in'] >= vent_time)].index)
    dataset[idx]=df
    print(dataset[idx].shape)

pid_list = dataset[0]['pid'].unique()
static_to_keep=static_to_keep[static_to_keep['pid'].isin(pid_list)]
static_covid=static_covid[static_covid['pid'].isin(pid_list)]

dataset[0].head()

(14824, 27)
(14824, 27)
(14824, 27)


,pid,hours_in,absolute_time,rr,hr,spo2,sbp,dbp,temp,hct,...,ph,hco3,po2,pco2,sao2,fio2,pfratio,age,gender_F,gender_M
0,300003674.0,0.0,17.0,18.0,92.0,98.0,137.0,76.0,36.5,41.5,...,7.400,24.0,87.5,40.0,97.0,0.21,416.666667,73,0,1
1,300003674.0,1.0,18.0,25.0,91.0,98.0,148.0,73.0,36.5,41.5,...,7.400,24.0,87.5,40.0,97.0,0.21,416.666667,73,0,1
2,300003674.0,2.0,19.0,25.0,85.0,97.0,139.0,74.0,37.4,41.5,...,7.409,19.9,51.3,31.1,86.8,0.90,57.000000,73,0,1
3,300003674.0,3.0,20.0,22.0,89.0,92.0,138.0,76.0,37.3,41.5,...,7.409,19.9,51.3,31.1,86.8,0.90,57.000000,73,0,1
4,300003674.0,4.0,21.0,16.0,96.0,96.0,160.0,87.0,37.5,41.5,...,7.409,19.9,51.3,31.1,86.8,0.90,57.000000,73,0,1


In [22]:
# 6시간 미만 체류 환자 제거
# ku-covid, ku-icu ==> 6시간보다 작은 환자 존재
for idx, df in enumerate(dataset):
    print(dataset[idx].shape)
    df = df.groupby('pid').filter(lambda x: x.shape[0] >= 6)
    ids = df['pid'].unique()
    static_covid =static_covid[static_covid['pid'].isin(ids)]
    static_to_keep =static_to_keep[static_to_keep['pid'].isin(ids)]
    dataset[idx] = df
    print(dataset[idx].shape)

dataset[0].head()


(14824, 27)
(14796, 27)
(14824, 27)
(14796, 27)
(14824, 27)
(14796, 27)


,pid,hours_in,absolute_time,rr,hr,spo2,sbp,dbp,temp,hct,...,ph,hco3,po2,pco2,sao2,fio2,pfratio,age,gender_F,gender_M
0,300003674.0,0.0,17.0,18.0,92.0,98.0,137.0,76.0,36.5,41.5,...,7.400,24.0,87.5,40.0,97.0,0.21,416.666667,73,0,1
1,300003674.0,1.0,18.0,25.0,91.0,98.0,148.0,73.0,36.5,41.5,...,7.400,24.0,87.5,40.0,97.0,0.21,416.666667,73,0,1
2,300003674.0,2.0,19.0,25.0,85.0,97.0,139.0,74.0,37.4,41.5,...,7.409,19.9,51.3,31.1,86.8,0.90,57.000000,73,0,1
3,300003674.0,3.0,20.0,22.0,89.0,92.0,138.0,76.0,37.3,41.5,...,7.409,19.9,51.3,31.1,86.8,0.90,57.000000,73,0,1
4,300003674.0,4.0,21.0,16.0,96.0,96.0,160.0,87.0,37.5,41.5,...,7.409,19.9,51.3,31.1,86.8,0.90,57.000000,73,0,1


In [23]:
# extract Y label
Y=dataset[0][['pid','hours_in']].copy()
Y['vent'] = 0

for subject_id in static_covid[static_covid['IHMV']==True]['pid']:
    vent_time = static_covid[static_covid['pid']==subject_id]['MV_time_'].item()
    idx = Y[(Y['pid']==subject_id) & (Y['hours_in'] >= vent_time-12)].index
    Y.loc[idx, 'vent'] = 1
    # print(Y.loc[idx, 'vent'])
Y.set_index(['pid','hours_in'],inplace=True)
Y.describe()

,vent
count,14796.000000
mean,0.023655
std,0.151977
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,1.000000


In [24]:
#set index
for idx, df in enumerate(dataset):
    df.set_index(['pid','hours_in'],inplace=True)
    dataset[idx]=df
dataset[0].columns

Index(['absolute_time', 'rr', 'hr', 'spo2', 'sbp', 'dbp', 'temp', 'hct', 'hb',
       'platelets', 'wbc', 'rbc', 'mch', 'mchc', 'mcv', 'ph', 'hco3', 'po2',
       'pco2', 'sao2', 'fio2', 'pfratio', 'age', 'gender_F', 'gender_M'],
      dtype='object')

In [25]:
def id_cols(df):
    df.rename(columns = {'pid':'subject_id'}, inplace = True)
    df.insert(1,'hadm_id', 0 )
    df.insert(2,'icustay_id', 0 )
    return df

for idx, df in enumerate(dataset):
    dataset[idx]=id_cols(df.reset_index()).set_index(['subject_id', 'hadm_id', 'icustay_id','hours_in'])
Y=id_cols(Y.reset_index()).set_index(['subject_id', 'hadm_id', 'icustay_id','hours_in'])
static_covid=id_cols(static_covid)
static_to_keep=id_cols(static_to_keep)

In [26]:
# 재입원자 1명 제거
# 추후 전처리에서 재입원 케이스 나누도록 해야할 듯
def drop_readmission(df):
    df = df.drop(df[(df['subject_id']==301981882)].index)
    return df

for idx, df in enumerate(dataset):
    dataset[idx] = drop_readmission(df.reset_index()).set_index(['subject_id', 'hadm_id', 'icustay_id','hours_in'])
    print(dataset[idx].shape)

Y = drop_readmission(Y.reset_index()).set_index(['subject_id', 'hadm_id', 'icustay_id','hours_in'])
static_covid = drop_readmission(static_covid)
static_to_keep =  drop_readmission(static_to_keep)
dataset[idx]

(14352, 25)
(14352, 25)
(14352, 25)


absolute_time    rr    hr   spo2  \
subject_id  hadm_id icustay_id hours_in                                     
300003674.0 0       0          0.0                17.0  18.0  92.0   98.0   
                               1.0                18.0  25.0  91.0   98.0   
                               2.0                19.0  25.0  85.0   97.0   
                               3.0                20.0  22.0  89.0   92.0   
                               4.0                21.0  16.0  96.0   96.0   
...                                                ...   ...   ...    ...   
302066555.0 0       0          221.0              20.0  20.0  71.0   97.0   
                               222.0              21.0  14.0  63.0  100.0   
                               223.0              22.0  16.0  54.0   99.0   
                               224.0              23.0  17.0  58.0   99.0   
                               225.0               0.0  17.0  65.0  100.0   

                                           sbp   dbp  temp   hct    hb  \
subject_id  hadm_id icustay_id hours_in                                  
300003674.0 0       0          0.0       137.0  76.0  36.5  41.5  13.5   
                               1.0       148.0  73.0  36.5  41.5  13.5   
                               2.0       139.0  74.0  37.4  41.5  13.5   
                               3.0       138.0  76.0  37.3  41.5  13.5   
                               4.0       160.0  87.0  37.5  41.5  13.5   
...                                        ...   ...   ...   ...   ...   
302066555.0 0       0          221.0      89.0  60.0  36.9  37.3  12.8   
                               222.0      92.0  65.0  36.2  37.3  12.8   
                               223.0      84.0  59.0  37.2  37.3  12.8   
                               224.0      83.0  50.0  37.2  37.3  12.8   
                               225.0      83.0  57.0  37.2  37.3  12.8   

                                         platelets  ...     ph  hco3   po2  \
subject_id  hadm_id icustay_id hours_in             ...                      
300003674.0 0       0          0.0           275.0  ...  7.400  24.0  87.5   
                               1.0           275.0  ...  7.400  24.0  87.5   
                               2.0           275.0  ...  7.409  19.9  51.3   
                               3.0           275.0  ...  7.409  19.9  51.3   
                               4.0           275.0  ...  7.409  19.9  51.3   
...                                            ...  ...    ...   ...   ...   
302066555.0 0       0          221.0         225.0  ...  7.400  24.0  87.5   
                               222.0         225.0  ...  7.400  24.0  87.5   
                               223.0         225.0  ...  7.400  24.0  87.5   
                               224.0         225.0  ...  7.400  24.0  87.5   
                               225.0         225.0  ...  7.400  24.0  87.5   

                                         pco2  sao2  fio2     pfratio  age  \
subject_id  hadm_id icustay_id hours_in                                      
300003674.0 0       0          0.0       40.0  97.0  0.21  416.666667   73   
                               1.0       40.0  97.0  0.21  416.666667   73   
                               2.0       31.1  86.8  0.90   57.000000   73   
                               3.0       31.1  86.8  0.90   57.000000   73   
                               4.0       31.1  86.8  0.90   57.000000   73   
...                                       ...   ...   ...         ...  ...   
302066555.0 0       0          221.0     40.0  97.0  0.21  416.666667   52   
                               222.0     40.0  97.0  0.21  416.666667   52   
                               223.0     40.0  97.0  0.21  416.666667   52   
                               224.0     40.0  97.0  0.21  416.666667   52   
                               225.0     40.0  97.0  0.21  416.666667   52   

                                         gende

In [27]:
vital = ['rr','hr','spo2','sbp','dbp','temp']
CBC = ['hct','hb','platelets','wbc','rbc','mch','mchc','mcv']
ABGA = ['ph','hco3', 'po2','pco2','sao2','fio2','pfratio']

dataset[0].reset_index()[ABGA].describe()

,ph,hco3,po2,pco2,sao2,fio2,pfratio
count,14352.000000,14352.000000,14352.000000,14352.000000,14352.000000,14352.000000,14352.000000
mean,7.439949,21.664932,91.524589,31.766116,96.266869,0.414053,259.726536
std,0.041544,3.318721,26.100138,6.086090,4.200652,0.168049,124.458632
min,6.766000,10.000000,28.800000,16.100000,48.600000,0.209000,34.800000
25%,7.421000,19.400000,75.700000,28.000000,95.700000,0.280000,159.200000
50%,7.442000,21.800000,87.500000,31.500000,97.000000,0.360000,249.900000
75%,7.463000,24.000000,100.200000,35.300000,98.200000,0.500000,334.400000
max,7.627000,43.000000,237.700000,200.000000,99.800000,1.000000,825.100000


In [28]:
static_covid = static_covid[['subject_id', 'hadm_id', 'icustay_id', 'icu_in', 'icu_out', 'age',
       'gender', 'MV_time', 'MV_time_', 'los', 'intime', 'IHMV']]

In [29]:
static_covid["MV_ICU"] = static_covid["IHMV"].astype(int)

In [30]:
import os.path

if DROPREF == False:
    DATAFILE = "../data/ku-covid/"
    name = ['ku_covid_worst.h5','ku_covid_mean.h5','ku_covid_first.h5']
    for idx, df in enumerate(dataset):
        path = os.path.join(DATAFILE,name[idx])
        print(path)
        df.to_hdf(path,'vitals_labs')
        Y.to_hdf(path,'interventions')
        static_covid.to_hdf(path,'patients', format='table')
        static_to_keep.to_hdf(path,'patients_to_keep', format='table')
else:
    DATAFILE = "../data/ku-covid/dropref/"
    name = ['ku_covid_worst.h5','ku_covid_mean.h5','ku_covid_first.h5']
    for idx, df in enumerate(dataset):
        path = os.path.join(DATAFILE,name[idx])
        print(path)
        df.to_hdf(path,'vitals_labs')
        Y.to_hdf(path,'interventions')
        static_covid.to_hdf(path,'patients', format='table')
        static_to_keep.to_hdf(path,'patients_to_keep', format='table')

../data/ku-covid/dropref/ku_covid_worst.h5
../data/ku-covid/dropref/ku_covid_mean.h5
../data/ku-covid/dropref/ku_covid_first.h5


In [31]:
DATAFILE = "../data/ku-covid/ku_covid_raw.h5"
X_raw.to_hdf(DATAFILE,'vitals_labs')
Y.to_hdf(DATAFILE,'interventions')
static_covid.to_hdf(DATAFILE,'patients', format='table')
static_to_keep.to_hdf(DATAFILE,'patients_to_keep', format='table')

In [32]:
import os.path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

DATAFILE_test = "../data/ku-covid/ku_covid_worst.h5"
X_test = pd.read_hdf(DATAFILE_test,'vitals_labs')
Y_test = pd.read_hdf(DATAFILE_test,'interventions')
static_test = pd.read_hdf(DATAFILE_test,'patients')
static_to_keep_test = pd.read_hdf(DATAFILE_test,'patients_to_keep')
X_test.describe()

,absolute_time,rr,hr,spo2,sbp,dbp,temp,hct,hb,platelets,...,ph,hco3,po2,pco2,sao2,fio2,pfratio,age,gender_F,gender_M
count,14382.000000,14382.000000,14382.000000,14382.000000,14382.000000,14382.000000,14382.000000,14382.000000,14382.000000,14382.000000,...,14382.000000,14382.000000,14382.000000,14382.000000,14382.000000,14382.000000,14382.000000,14382.000000,14382.000000,14382.000000
mean,11.573286,20.176749,74.493464,96.012794,122.208872,74.201015,36.722987,36.467313,12.294368,256.080865,...,7.440949,21.565672,91.901272,31.487095,96.255994,0.420071,256.040145,62.368238,0.324781,0.675219
std,6.910427,5.054097,18.951596,4.017240,20.039198,13.263558,0.470932,5.942183,2.032647,116.968689,...,0.041378,3.351004,26.777398,5.975483,4.231882,0.167997,123.840785,15.546695,0.468309,0.468309
min,0.000000,0.000000,3.000000,3.000000,51.000000,32.000000,34.500000,19.100000,6.400000,24.000000,...,6.766000,10.000000,28.800000,16.100000,48.600000,0.209000,34.800000,26.000000,0.000000,0.000000
25%,6.000000,17.000000,60.000000,95.000000,109.000000,65.000000,36.400000,33.000000,11.100000,164.000000,...,7.423000,19.400000,75.300000,27.800000,95.600000,0.290000,156.700000,49.000000,0.000000,0.000000
50%,12.000000,20.000000,71.000000,97.000000,121.000000,73.000000,36.700000,37.000000,12.400000,249.000000,...,7.442000,21.600000,87.500000,31.300000,97.000000,0.400000,243.600000,64.000000,0.000000,1.000000
75%,18.000000,23.000000,85.000000,99.000000,135.000000,83.000000,37.000000,40.300000,13.700000,331.000000,...,7.464000,24.000000,101.800000,35.000000,98.200000,0.550000,325.600000,74.000000,1.000000,1.000000
max,23.000000,73.000000,197.000000,100.000000,367.000000,179.000000,39.900000,50.200000,17.400000,669.000000,...,7.627000,43.000000,237.700000,200.000000,99.800000,1.000000,825.100000,96.000000,1.000000,1.000000
